# decorator

まずはシンプルなデコレータ

In [1]:
def deco(f):
    def inner(*args, **kwargs):
        print('---before---')
        result = f(*args, **kwargs)
        print('---after---')
        return result
    return inner

In [2]:
def greet(name):
    print('hello, {0}'.format(name))

In [3]:
greet('masa')

hello, masa


In [4]:
deco_greet = deco(greet)

In [5]:
deco_greet('masa')

---before---
hello, masa
---after---


@はsyntax sugar

In [6]:
@deco
def another_deco_greet(name):
    print('hello, {0}'.format(name))

In [7]:
another_deco_greet('masa')

---before---
hello, masa
---after---


# instance method を decorate

In [8]:
class Human:
    def __init__(self, name):
        self.name = name
    
    def greet(self):
        print('my name is {0}'.format(self.name))

In [9]:
masa = Human('masa')
masa.greet()

my name is masa


In [12]:
def method_deco(f):
    def inner(*args, **kwargs):
        print('---before---')
        print(args, kwargs)
        result = f(*args, **kwargs)
        print('---after---')
        return result
    return inner

class DecoHuman:
    def __init__(self, name):
        self.name = name
    
    @method_deco
    def greet(self):
        print('my name is {0}'.format(self.name))

In [13]:
masa = DecoHuman('masa')
masa.greet()

---before---
(<__main__.DecoHuman object at 0x104bd3208>,) {}
my name is masa
---after---


- args に `DecoHuman` のinstanceが渡ってることがわかる
- 要するに `greet(self)` における `self` のこと．

# decorator class

- decorator は function ではなく class としても書ける

In [17]:
class Deco:
    def __init__(self, func):
        self.func = func
        
    def __call__(self, *args, **kwargs):
        print('---before---')
        print(self, args, kwargs)
        result = self.func(*args, **kwargs)
        print('---after---')
        return result

In [18]:
@Deco
def hello(name):
    print('hello {0}'.format(name))

In [19]:
hello('masa')

---before---
<__main__.Deco object at 0x104bc86a0> ('masa',) {}
hello masa
---after---


# decorator class で instance method を decorate

In [21]:
class Deco:
    def __init__(self, func):
        self.func = func
        
    def __call__(self, *args, **kwargs):
        print('---before---')
        print(self, args, kwargs)
        result = self.func(*args, **kwargs)
        print('---after---')
        return result

class DecoHuman:
    def __init__(self, name):
        self.name = name
    
    @Deco
    def greet(self):
        print('my name is {0}'.format(self.name))

In [22]:
masa = DecoHuman('masa')
masa.greet()

---before---
<__main__.Deco object at 0x104bb29e8> () {}


TypeError: greet() missing 1 required positional argument: 'self'

- positional argument の self がない...だと...
- decorate された method からはownerであるclass instance を取れない
    - inctanceに対する呼び出しのときはキチンとselfが挟まれるっぽい
    - class化するとそうでもないっぽい
    - functionとして書いているときは self 意識してないのに...??
- pythonにおいて instance method 呼ばれるとき self がどう渡されるか？がポイントっぽい
    - method呼ぶときは外部から見ると self が部分適用されてる形のはず
    - funcでの decorator は 部分適用後の関数をdecotate: bound method を decorate
        - あくまで1関数としてcallされる -> デフォルトの__call__なり__get__なりが適用されそう
    - classでの decorator は 部分適用前の関数をdecorate: unbound method を decorate
        - Deco objectの call が呼ばれる
    - と考えるとわかりやすいか
- deoctate する時点で `self` の参照渡せてないとイケない..??
    - いや関数からそれ見るの無理では...
- 関係ありそうなエントリ
    - http://coreblog.org/ats/translation-of-why-explicit-self-has-to-stay/
    - http://stackoverflow.com/questions/10401935/python-method-wrapper-type
    - https://docs.python.jp/3/howto/descriptor.html#functions-and-methods
- 結論
    - decoratorをclassで書くって方針がアレでは．．．

In [32]:
class Deco:
    def __init__(self, func):
        self.func = func
        
    def __call__(self, *args, **kwargs):
        print('---before---')
        print(self, args, kwargs)
        result = self.func(*args, **kwargs)
        print('---after---')
        return result

class DecoHuman:
    def __init__(self, name):
        self.name = name
    
    @Deco
    def greet(self):
        print('my name is {0}'.format(self.name))
        
masa = DecoHuman('masa')
print(type(DecoHuman.greet.func))
print(type(masa.greet.func))

<class 'function'>
<class 'function'>


In [34]:
class Deco:
    def __init__(self, func):
        print(func)
        self.func = func
        
    def __call__(self, *args, **kwargs):
        print('---before---')
        print(self, args, kwargs)
        result = self.func(*args, **kwargs)
        print('---after---')
        return result

class DecoHuman:
    def __init__(self, name):
        self.name = name
    
    def greet(self):
        print('my name is {0}'.format(self.name))
        
masa = DecoHuman('masa')
masa.greet = Deco(masa.greet)
masa.greet()

<bound method DecoHuman.greet of <__main__.DecoHuman object at 0x105478588>>
---before---
<__main__.Deco object at 0x105478518> () {}
my name is masa
---after---


In [36]:
import types

class Deco:
    def __init__(self, func):
        self.func = func
    
    def __get__(self, obj, objtype=None):
        "Simulate func_descr_get() in Objects/funcobject.c"
        return types.MethodType(self, obj, objtype)
        
    def __call__(self, *args, **kwargs):
        print('---before---')
        print(self, args, kwargs)
        result = self.func(*args, **kwargs)
        print('---after---')
        return result

class DecoHuman:
    def __init__(self, name):
        self.name = name
    
    @Deco
    def greet(self):
        print('my name is {0}'.format(self.name))
        
masa = DecoHuman('masa')
masa.greet()

TypeError: method expected 2 arguments, got 3

pythonにおける「関数オブジェクト」がどんな振る舞いするのか，がポイントぽいぞ
- https://docs.python.jp/3/howto/descriptor.html#static-methods-and-class-methods

object経由で呼ばれるときはよしなに部分適用してくれるっぽさがある

てことは functionの基底クラスをパクってdecoratorにすればワンチャン
...いやそこまで頑張ってfunctionオブジェクトの挙動を真似するより，純粋にfunctionとして書いたほうがいいよねこれ．．．

なんとかして部分適用する部分かければいいんだけどねー...
- https://docs.python.jp/3/library/functools.html#functools.partialmethod

こいつ使えそう？

In [38]:
import functools

class Deco:
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args, **kwargs):
        print('---before---')
        print(self, args, kwargs)
        result = self.func(*args, **kwargs)
        print('---after---')
        return result

class DecoHuman:
    def __init__(self, name):
        self.name = name
    
    @functools.partialmethod
    @Deco
    def greet(self):
        print('my name is {0}'.format(self.name))
        
masa = DecoHuman('masa')
masa.greet()

---before---
<__main__.Deco object at 0x1054aee10> (<__main__.DecoHuman object at 0x1054aee80>,) {}
my name is masa
---after---


きたぁぁぁぁぁぁぁ